In [1]:
import duckdb
import numpy as np
from transformers import BigBirdModel, BigBirdTokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gc

2023-10-10 22:33:53.690258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [12]:
!pip update duckdb

ERROR: unknown command "update"


In [3]:
con.execute("CREATE TABLE BigBird(id INT PRIMARY KEY, data FLOAT8[]);")

In [4]:
con.execute("CREATE TABLE BigBird_norm(id INT UNIQUE REFERENCES BigBird(id), norm FLOAT8);")

In [5]:
con.execute("SET max_expression_depth = 8192;")

ConnectionException: Connection Error: Connection has already been closed

In [12]:
concedendo_label = 1
negando_label = 0

In [5]:
data = con.execute("SELECT ID,summarize FROM decisions_description where summarize not null ").fetchnumpy()

In [6]:
data["ID"]

array([121184492, 122027196, 123312824, 123447928, 123917739, 124072953,
       125636167, 127181699, 127506614, 127549781, 127550136, 128070511,
       128814277, 129636667, 129874810, 129980126, 130056982, 130072596,
       130148392, 130223439, 130527099, 130933413, 131106904, 131114240,
       131460066, 132206882, 132398032, 133166046, 133275190, 133344745,
       134122233, 134322476, 134322487, 134418450, 134480749, 134871679,
       134995474, 135074005, 135285046, 135826326, 135973386, 136063008,
       136098974, 136355932, 136478873, 136662016, 136869975, 136877456,
       136887676, 136939921, 137027671, 137063080, 137070067, 137183368,
       137190449, 137195556, 137235849, 137260110, 137263698, 137317536,
       137444602, 137444616, 137470368, 137720157, 137883023, 137883071,
       137903248, 138013755, 138018966, 138058976, 138060974, 138126797,
       138131375, 138154814, 138219087, 138221040, 138222032, 138274548,
       138370598, 138403297, 138446675, 138487953, 

In [7]:
data["summarize"].data

array(['Gulliver argues that the prison system is inherently precarious because it lacks adequate food, medical care, and proper sanitation. The patient, Valter de Albano, has requested for conversion from prison to home, but the prison authorities have rejected his request due to the absence of legally required requirements. Furthermore, the patient\'s condition is not as dire as those described in the national recommendation for release.\nthe authorities decide to take measures against the new encephalovirus in order to reduce the risk of spread and to ensure that the population does not get sick. The authorities also decide to grant house arrest to persons arrested in accordance with the provisions of the Articles of Confederation. According to the news, the prison has good medical assistance and hygiene conditions, but there is no communication from the IoC-19 in the cell. Furthermore, the patient\'s age is young and not in the "age range of risk." This means that the authorities n

In [8]:
data["ID"].size

800

In [9]:
tokenizer = BigBirdTokenizerFast.from_pretrained("google/bigbird-roberta-base") # large
model = BigBirdModel.from_pretrained("google/bigbird-roberta-base",
    #attention_type="block_sparse"
)

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=4096, chunk_overlap=10 #86683 4096
)

In [11]:
for index,e in enumerate(data["ID"]):
  print(index,e)
  #texts = text_splitter.split_text(data["summarize"][index])
  #print(index,len(texts))


  inputs = tokenizer(data["summarize"][index], return_tensors="pt")
  outputs = model(**inputs)

  last_hidden_states = outputs.last_hidden_state
  print(index,last_hidden_states[0].shape)

  vetor = last_hidden_states[0].flatten().detach().numpy()
  con.execute('INSERT INTO BigBird (id,data) VALUES (?,?)', (int(e),vetor))


0 121184492
0 torch.Size([211, 768])
1 122027196
1 torch.Size([219, 768])
2 123312824
2 torch.Size([322, 768])
3 123447928
3 torch.Size([213, 768])
4 123917739
4 torch.Size([465, 768])
5 124072953
5 torch.Size([170, 768])
6 125636167
6 torch.Size([137, 768])
7 127181699
7 torch.Size([589, 768])
8 127506614
8 torch.Size([1179, 768])
9 127549781
9 torch.Size([538, 768])
10 127550136
10 torch.Size([335, 768])
11 128070511
11 torch.Size([80, 768])
12 128814277
12 torch.Size([85, 768])
13 129636667
13 torch.Size([3125, 768])
14 129874810
14 torch.Size([111, 768])
15 129980126
15 torch.Size([278, 768])
16 130056982
16 torch.Size([231, 768])
17 130072596
17 torch.Size([415, 768])
18 130148392
18 torch.Size([142, 768])
19 130223439
19 torch.Size([298, 768])
20 130527099
20 torch.Size([114, 768])
21 130933413
21 torch.Size([73, 768])
22 131106904
22 torch.Size([492, 768])
23 131114240
23 torch.Size([288, 768])
24 131460066
24 torch.Size([486, 768])
25 132206882
25 torch.Size([76, 768])
26 13239

In [5]:
con.execute("""
WITH norms AS (
  SELECT 
      id,
      SQRT(SUM(POWER(val, 2))) AS norm
  FROM 
      BigBird,
      LATERAL UNNEST(data) AS t(val)
  GROUP BY 
      id
)
INSERT INTO BigBird_norm (id, norm)
SELECT id, norm
FROM norms

ON CONFLICT (id) 
DO UPDATE SET norm = EXCLUDED.norm; 
""").fetchnumpy()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

{'Count': masked_array(data=[604],
              mask=[False],
        fill_value=999999)}

In [12]:
con.execute("SELECT * FROM BigBird_norm").df()

,id,norm
0,121184492,168.704783
1,122027196,171.267925
2,123312824,196.774132
3,123447928,163.537986
4,123917739,218.952587
...,...,...
599,141823626,95.057622
600,141846106,158.961095
601,141849401,216.833187
602,142302318,173.251509


In [13]:
gc.collect()

492

In [14]:
data = con.execute("""
WITH DotProducts AS (
    WITH expanded AS (
        SELECT
            a.id AS idA,
            b.id AS idB,
            unnest(a.data) AS a_data,
            unnest(b.data) AS b_data,
            row_number() OVER(PARTITION BY a.id, b.id ORDER BY a.id) as rn
        FROM
            BigBird a
        CROSS JOIN
            BigBird b
        WHERE
            a.id < b.id
    )

    SELECT
        idA,
        idB,
        sum(a_data * b_data) AS dot_product
    FROM
        expanded
    GROUP BY
        idA, idB
                
    )
SELECT 
    d.idA,
    d.idB,
    CASE 
        WHEN n1.norm = 0 OR n2.norm = 0 THEN 0
        ELSE d.dot_product / (n1.norm * n2.norm)
    END AS cosine_similarity
FROM 
    DotProducts d
JOIN 
    BigBird_norm n1 ON d.idA = n1.id
JOIN 
    BigBird_norm n2 ON d.idB = n2.id;
""")          

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
data.df()

{'idA': array([], dtype=int32),
 'idB': array([], dtype=int32),
 'cosine_similarity': array([], dtype=float64)}

In [6]:
con.execute("drop table BigBird2_norm")

In [7]:
con.execute("drop table BigBird")

In [11]:
con.close()